# Simulator data generation. 
/ Annotated files from https://drive.google.com/file/d/0B-Eiyn-CUQtxdUZWMkFfQzdObUE/view?usp=sharing

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import yaml
import imageio
from object_detection.utils import dataset_util

In [2]:
sess=tf.InteractiveSession()
%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
boxes = yaml.load(open('/home/amol/Downloads/sim_data/data/sim_training_data/sim_data_annotations.yaml','r'))

In [5]:
boxes[1]

{'annotations': [{'class': 'Yellow',
   'x_width': 48.0,
   'xmin': 130.24,
   'y_height': 120.32,
   'ymin': 288.32},
  {'class': 'Yellow',
   'x_width': 47.039999999999964,
   'xmin': 377.6,
   'y_height': 115.51999999999998,
   'ymin': 294.72},
  {'class': 'Yellow',
   'x_width': 48.319999999999936,
   'xmin': 625.6,
   'y_height': 119.04000000000002,
   'ymin': 299.52}],
 'class': 'image',
 'filename': 'sim_data_capture/left0011.jpg'}

In [6]:
label_set = set()
for box in boxes:
    for boxi in box['annotations']:
        label_set.add(boxi['class'].encode())

In [7]:
label_set

{b'Green', b'Red', b'Yellow'}

In [8]:
def validate_bbox(box_info):
    vec = [box_info['x_max'],box_info['x_min'],box_info['y_max'],box_info['y_min']]
    check1 = any([(not np.isnan(i)) and (i>0) for i in vec])
    check2 = (box_info['x_max']<=800) and (box_info['x_min']<=800) and (box_info['x_max']>=box_info['x_min'])
    check3 = (box_info['y_max']<=600) and (box_info['y_min']<600) and (box_info['y_max']>=box_info['y_min'])
    return check1 and check2 and check3

In [12]:
def extract_box_info(boxes_dict):
    result_box_info = []
    labels = []
    for box_info in boxes_dict:
        box_info['x_min']= box_info['xmin']
        box_info['x_max'] = box_info['x_min']+box_info['x_width']
        box_info['y_min']= box_info['ymin']
        box_info['y_max'] = box_info['y_min']+box_info['y_height']
        box_info['label'] = box_info['class']
        if validate_bbox(box_info):
            result_box_info.append(np.array([box_info['x_max'],box_info['x_min'],box_info['y_max'],box_info['y_min']]))
            labels.append(box_info['label'].encode())
        else:
            return None 
    result_box_info = np.array(result_box_info)
    if(len(labels)==0):
        return {'x_max':np.array([],dtype=np.float32),'x_min':np.array([],dtype=np.float32),'y_min':np.array([],dtype=np.float32),'y_max':np.array([],dtype=np.float32),'labels':labels}
    return {'x_max':result_box_info[:,0],'x_min':result_box_info[:,1],'y_max':result_box_info[:,2],'y_min':result_box_info[:,3],'labels':labels}


In [13]:
a=extract_box_info(boxes[3]['annotations'])

In [14]:
a

{'labels': [b'Yellow', b'Yellow', b'Yellow'],
 'x_max': array([174.08, 426.88, 680.32]),
 'x_min': array([126.72, 376.32, 625.92]),
 'y_max': array([407.36, 413.44, 418.88]),
 'y_min': array([285.44, 291.2 , 294.72])}

In [15]:
LABELMAP = dict(zip(list(label_set),range(1,1+len(label_set))))#not sure if id should start from one but just to be safe

In [16]:
def create_tf_example(filename,boxes):
    """Creates a tf.Example proto from sample cat image.

    Args:
        encoded_image_data: The jpg encoded data of the cat image.

    Returns:
        example: The created tf.Example.
    """

    height = 600
    width = 800
    image_format = b'jpg'

    xmins = np.array([i/width for i in boxes['x_min']])
    xmaxs = np.array([i/width for i in boxes['x_max']])
    ymins = np.array([i/height for i in boxes['y_min']])
    ymaxs = np.array([i/height for i in boxes['y_max']])
    classes_text = boxes['labels']
    classes = [LABELMAP[i] for i in classes_text]
    with tf.gfile.GFile(filename,'rb') as fid:
        encoded_image_data = fid.read()
    tf_example = tf.train.Example(features=tf.train.Features(feature={
      'image/height': dataset_util.int64_feature(height),
      'image/width': dataset_util.int64_feature(width),
      'image/filename': dataset_util.bytes_feature(filename.encode()),
      'image/source_id': dataset_util.bytes_feature(filename.encode()),
      'image/encoded': dataset_util.bytes_feature(encoded_image_data),
      'image/format': dataset_util.bytes_feature(image_format),
      'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
      'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
      'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
      'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
      'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
      'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

In [19]:
t = create_tf_example('/home/amol/Downloads/sim_data/data/sim_training_data/'+ boxes[5]['filename'],extract_box_info(boxes[5]['annotations']))

In [26]:
a

{'labels': [b'Yellow', b'Yellow', b'Yellow'],
 'x_max': array([174.08, 426.88, 680.32]),
 'x_min': array([126.72, 376.32, 625.92]),
 'y_max': array([407.36, 413.44, 418.88]),
 'y_min': array([285.44, 291.2 , 294.72])}

In [27]:
import contextlib2
from object_detection.dataset_tools import tf_record_creation_util

num_shards=2
output_filebase='/home/amol/ObjectDetection/SimulatorData/train_'

writer = tf.python_io.TFRecordWriter
with contextlib2.ExitStack() as tf_record_close_stack:
    output_tfrecords = tf_record_creation_util.open_sharded_output_tfrecords(
      tf_record_close_stack, output_filebase, num_shards)
    index = 0
    for box_dict in boxes[0:200]:
        boxes_per_image = extract_box_info(box_dict['annotations'])
        if boxes_per_image:
            path ='/home/amol/Downloads/sim_data/data/sim_training_data/'+box_dict['filename']
            tf_example = create_tf_example(path,boxes_per_image)
            output_shard_index = index % num_shards
            output_tfrecords[output_shard_index].write(tf_example.SerializeToString())
            index += 1

In [28]:
import contextlib2
from object_detection.dataset_tools import tf_record_creation_util

num_shards=2
output_filebase='/home/amol/ObjectDetection/SimulatorData/test_'

writer = tf.python_io.TFRecordWriter
with contextlib2.ExitStack() as tf_record_close_stack:
    output_tfrecords = tf_record_creation_util.open_sharded_output_tfrecords(
      tf_record_close_stack, output_filebase, num_shards)
    index = 0
    for box_dict in boxes[200:]:
        boxes_per_image = extract_box_info(box_dict['annotations'])
        if boxes_per_image:
            path ='/home/amol/Downloads/sim_data/data/sim_training_data/'+box_dict['filename']
            tf_example = create_tf_example(path,boxes_per_image)
            output_shard_index = index % num_shards
            output_tfrecords[output_shard_index].write(tf_example.SerializeToString())
            index += 1

In [29]:
with open('/home/amol/ObjectDetection/SimulatorData/label_map.pbtxt','w') as fid:
    for key in LABELMAP.keys():
        output_string = "item{{id:{id}\n name:'{name}' }}"
        fid.write(output_string.format(id=LABELMAP[key],name=key.decode('utf-8')))
        fid.write('\n\n')
    